In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import sys
sys.path.append('../src')
from torch import nn
from omegaconf import OmegaConf
from torch.utils.data import DataLoader
import torch
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
import numpy as np
from tqdm import tqdm
pd.options.display.max_columns=100

In [2]:
from datalib import VentilatorData,VentilatorData2,VentilatorDataClassification
import modellib
from utils import fc
import datalib

In [18]:
m = torch.stack([torch.tensor([[1,2,2,3,4,4,3,2,2,23,4]])]*5)

In [19]:
m.shape

torch.Size([5, 1, 11])

In [34]:
# m = torch.stack([torch.tensor(np.ones((32,5)))]*4)
m = torch.stack([torch.tensor(np.ones(32))]*4)
m.shape

torch.Size([4, 32])

In [36]:
torch.hstack([x for x in m]).shape

torch.Size([128])

In [3]:
DATA_DIR = '/mnt/disks/extra_data/kaggle/ventilator_prediction/'
R_MAP = {5: 0, 50: 1, 20: 2}
C_MAP = {20: 0, 50: 1, 10: 2}

In [4]:
train = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))
test = pd.read_csv(os.path.join(DATA_DIR,'test.csv'))
train.breath_id = train.breath_id.map(dict(zip(train.breath_id.unique().tolist(),range(train.breath_id.nunique()))))
test.breath_id = train.breath_id.map(dict(zip(train.breath_id.unique().tolist(),range(train.breath_id.nunique()))))

In [5]:
pressure_dict = dict(zip(train['pressure'].unique().tolist(),range(train['pressure'].nunique())))
pressure_reverse_dict = {v:k for k,v in pressure_dict.items()}

In [6]:
train['pressure'] = train['pressure'].map(pressure_dict)

In [7]:
train = fc(train)

5
10
5
10


In [8]:
train.R = train.R.map(R_MAP)
train.C = train.C.map(C_MAP)

In [9]:
group_dict={}
for i,x in tqdm(train.groupby('breath_id'),total=train.breath_id.nunique()):
    m = x.to_dict(orient='records')
    group_dict[i] = {k:[a[k] for a in m] for k in [k for k in m[0].keys() if k not in ['id','breath_id']]}

100%|██████████| 75450/75450 [08:52<00:00, 141.75it/s]


In [10]:
config = OmegaConf.load('../configs/lstm-classification.yaml')
config.model.kwargs["output_dim"] = 950

In [11]:
from litmodellib import ClassifcationModel

In [12]:
lit_model = ClassifcationModel(
    config,
    num_train_iter=1,
    num_steps=1,
    mapping='../pressure_mapper.pkl',
    
)

{'init_type': 'yakama'}
tensor(111.3332)
tensor(25.3399)


In [13]:
data = getattr(datalib,config.dataset.train['class'])(**config.dataset.train['kwargs'],group_dict=group_dict)
dl = DataLoader(
    dataset=data,
    shuffle=True,
    num_workers=1,
    batch_size=8,
    pin_memory=True,
)

In [14]:
m = next(iter(dl))

In [15]:
mod = getattr(modellib,config.model['class'])(config.model['kwargs'])

{'init_type': 'yakama'}
tensor(-92.4493)
tensor(-201.7668)


In [16]:
preds = mod(m)

In [17]:
preds = preds.view(-1,950)

In [18]:
preds.shape

torch.Size([640, 950])

torch.Size([8, 80])

In [21]:
m1 = torch.stack([preds]*4)
m2 = torch.stack([m['target'].view(-1)]*4)

In [22]:
m1.shape,m2.shape

(torch.Size([4, 640, 950]), torch.Size([4, 640]))

In [24]:
torch.vstack([x for x in m1]).shape

torch.Size([2560, 950])

In [25]:
torch.hstack([x for x in m2]).shape

torch.Size([2560])

In [68]:
# m1 = torch.cat(m1,dim=0)
m2 = torch.cat(m2,dim=0)

In [36]:
def get_true_val(mapping,tensor):
    return torch.tensor([mapping[x.item()] for x in tensor])    

In [40]:
%%time
topk = m1.topk(k=3, dim=1).indices
true_preds = (
    get_true_val(mapping, topk.view(-1))
    .view(-1, 3)
    .median(dim=1)
    .values
)


CPU times: user 767 ms, sys: 0 ns, total: 767 ms
Wall time: 364 ms


In [69]:
m2.shape

torch.Size([20480])

In [37]:
import joblib
mapping = joblib.load('../pressure_mapper.pkl')

In [99]:
config.model.kwargs['output_dim'] = 950

In [100]:
config.model.kwargs['output_dim']

950

In [101]:
%%time
k=1
topk = preds.view(-1,950).topk(k=k,dim=1).indices
torch.tensor([torch.median(get_true_val(mapping,x)).item() for x in topk]).mean()

CPU times: user 116 ms, sys: 261 µs, total: 116 ms
Wall time: 24.2 ms


tensor(33.7162)

In [85]:
%%time
_ = get_true_val(mapping,top3.view(-1)).view(-1,3)

CPU times: user 7.75 ms, sys: 3.86 ms, total: 11.6 ms
Wall time: 10.9 ms


In [103]:
%%time
get_true_val(mapping,topk.view(-1)).view(-1,k).median(dim=1).values.mean()

CPU times: user 792 µs, sys: 4.05 ms, total: 4.84 ms
Wall time: 4.35 ms


tensor(33.7162)

In [59]:
%%time
get_true_val(mapping,preds.view(-1,950).argmax(1)).mean()

CPU times: user 12.6 ms, sys: 0 ns, total: 12.6 ms
Wall time: 3.93 ms


tensor(33.7162)

In [37]:
nn.CrossEntropyLoss()(preds.view(-1,950),m['target'].view(-1))

tensor(6.8633, grad_fn=<NllLossBackward>)

In [42]:
def get_true_val(mapping,tensor):
    return torch.tensor([mapping[x.item()] for x in tensor])    

In [62]:
# get_true_val(pressure_reverse_dict,m['target'].view(-1))

In [81]:
true_preds = get_true_val(pressure_reverse_dict,preds.argmax(2).view(-1))

In [84]:
[preds.argmax(2).view(-1).numpy()]

array([851, 143, 906, 168, 906, 906, 869, 757, 906, 906, 906, 906, 843,
       516, 843, 906, 843, 662, 143, 906, 906, 869, 869, 906, 906, 168,
       843, 280, 906, 143, 168, 843, 843, 906, 843, 906, 843, 280, 843,
       906, 280, 168, 869, 906, 906, 143, 906, 843, 711, 168, 280, 869,
       843, 280, 168, 906, 869, 823, 168, 843, 906, 823, 906, 168, 516,
       869, 280, 906, 851, 143, 843, 906, 143, 869, 869, 906, 143, 143,
       906, 280, 143, 168, 143, 906, 168, 906, 906, 906, 906, 906, 843,
       906, 168, 869, 168, 869, 843, 175, 869, 843, 906, 869, 906, 906,
       906, 843, 869, 906, 843, 906, 861, 906, 906, 168, 869, 168, 843,
       869, 843, 906, 843, 143, 168, 843, 843, 843, 168, 659, 869, 143,
       906, 869, 143, 859, 280, 143, 906, 906, 869, 168, 843, 906, 843,
       906, 143, 168, 869, 143, 843, 143, 143, 843, 143, 843, 906, 869,
       823, 843, 843, 143, 364, 143, 168, 906, 168, 906, 143, 843, 906,
       168, 843, 851, 869, 906, 168, 851, 843, 662, 168, 906, 17

In [ ]:
get_true_val(pressure_reverse_dict,preds.argmax(2).view(-1))

In [86]:
preds.view(-1,950).argmax(1)

tensor([851, 143, 906, 168, 906, 906, 869, 757, 906, 906, 906, 906, 843, 516,
        843, 906, 843, 662, 143, 906, 906, 869, 869, 906, 906, 168, 843, 280,
        906, 143, 168, 843, 843, 906, 843, 906, 843, 280, 843, 906, 280, 168,
        869, 906, 906, 143, 906, 843, 711, 168, 280, 869, 843, 280, 168, 906,
        869, 823, 168, 843, 906, 823, 906, 168, 516, 869, 280, 906, 851, 143,
        843, 906, 143, 869, 869, 906, 143, 143, 906, 280, 143, 168, 143, 906,
        168, 906, 906, 906, 906, 906, 843, 906, 168, 869, 168, 869, 843, 175,
        869, 843, 906, 869, 906, 906, 906, 843, 869, 906, 843, 906, 861, 906,
        906, 168, 869, 168, 843, 869, 843, 906, 843, 143, 168, 843, 843, 843,
        168, 659, 869, 143, 906, 869, 143, 859, 280, 143, 906, 906, 869, 168,
        843, 906, 843, 906, 143, 168, 869, 143, 843, 143, 143, 843, 143, 843,
        906, 869, 823, 843, 843, 143, 364, 143, 168, 906, 168, 906, 143, 843,
        906, 168, 843, 851, 869, 906, 168, 851, 843, 662, 168, 9

In [65]:
preds.shape

torch.Size([8, 80, 950])